In [1]:
from miss_mech import *
%load_ext autoreload
%autoreload 2


In [2]:
import torch
import os
import sklearn

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
d_lst = os.listdir('../data_proc')

FileNotFoundError: [Errno 2] No such file or directory: '../data_proc'

In [4]:
## NEW PROC

for id_ in range(len(d_lst)):
    
    
    dataset, X_num, X_cat, categories, d_numerical, y, task_type, _, _ = torch.load(f'../data_proc/{d_lst[id_]}').values()

    save_dir = f'../amputation/{dataset}'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    print(dataset)
    if (X_cat is not None):
        if (X_num is not None):
            print(f'Mixed data; num: {X_num.shape[1]}, cat: {X_cat.shape[1]} ')
    
            normalizer = sklearn.preprocessing.QuantileTransformer(
                                                output_distribution='normal',
                                                n_quantiles=max(min(X_num.shape[0] // 30, 1000), 10),
                                                subsample=int(1e9),
                                                random_state=42,
                                                )
            X_num = torch.tensor(normalizer.fit(X_num).transform(X_num)).float()
            
            X_init = torch.cat([X_num, X_cat], axis = 1)
            
        else:
            print(f'Categorical data; cat: {X_cat.shape[1]}')
            X_init = X_cat
                    
    else:
        print(f'Numerical data; num: {X_num.shape[1]}')
        
        normalizer = sklearn.preprocessing.QuantileTransformer(
                                                output_distribution='normal',
                                                n_quantiles=max(min(X_num.shape[0] // 30, 1000), 10),
                                                subsample=int(1e9),
                                                random_state=42,
                                                )
        X_num = torch.tensor(normalizer.fit(X_num).transform(X_num)).float()
    
        X_init = X_num

    
    simul_dict = {'X_init': X_init, 'X_num':X_num, 'X_cat': X_cat, 'y':y, 'task_type':task_type, 'd_numerical': d_numerical}
    torch.save(simul_dict, f'{save_dir}/new_proc.pkl')

    data_dict = {'MAR':{'0.3':{}, '0.6': {}}, 
                 'MCAR':{'0.3':{}, '0.6': {}}, 
                 'MNAR': {'0.3':{}, '0.6': {}}}
    
    
    for mechanism in ['MAR', 'MCAR', 'MNAR']:
        
        
        p_miss_r = 0.5
        
        for p_miss_c in [0.3, 0.6]:

            data_dict[mechanism][p_miss_c] = {}
            
            for seed in range(5):

                data_dict[mechanism][p_miss_c][seed] = {}
                
                
                #seed = 1
                print(mechanism)
                
                set_all_seeds(seed)
                
                # generate mask (missing); True : Miss, False: Obs
                ps, mask, coeffs = generate_missing(X_num, X_cat, mechanism = mechanism, p = p_miss_r, p_obs_col = 1-p_miss_c, seed = seed)
                
                #print( (1.0*mask).mean(axis = 0) )
                
                X_incomp = X_init.clone().float()
                X_incomp[mask.bool()] = np.nan
                
                X_incomp_num, X_incomp_cat = X_incomp[:, :d_numerical], X_incomp[:, d_numerical:]
                miss_cols = torch.where(mask.float().mean(axis = 0) != 0)[0].tolist()
                fully_obs_cols = [i for i in range(X_init.shape[1]) if i not in miss_cols]

                
                base_dict = {'X_incomp': X_incomp,
                             'mask': mask,
                             'full_cols': fully_obs_cols,
                             'ps': ps}

                data_dict[mechanism][p_miss_c][seed] = base_dict

    # SAVE
    torch.save(data_dict, f'{save_dir}/amputed.pkl')


raisin
Numerical data; num: 7
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
wine
Numerical data; num: 11
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
climate
Mixed data; num: 19, cat: 1 
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
yacht
Categorical data; cat: 6
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
adult
Mixed data; num: 6, cat: 9 
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
MNAR
default
Mixed data; num: 14, cat: 10 
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MCAR
MNAR
MNAR
M

In [ ]:
## ORG PROC

from miss_mech_rem import *
from sklearn.preprocessing import MinMaxScaler

def scale_data(X: pd.DataFrame):
    preproc = MinMaxScaler()
    cols = X.columns
    return pd.DataFrame(preproc.fit_transform(X), columns=cols)
    
## ORG PROC

for id_ in range(len(d_lst)):
    
    
    dataset, X_num, X_cat, categories, d_numerical, y, task_type, _, _ = torch.load(f'../data_proc/{d_lst[id_]}').values()

    save_dir = f'../amputation/{dataset}'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    print(dataset)
    if (X_cat is not None):
        if (X_num is not None):
            print(f'Mixed data; num: {X_num.shape[1]}, cat: {X_cat.shape[1]} ')
            X_init = torch.cat([X_num, X_cat], axis = 1)
            
        else:
            print(f'Categorical data; cat: {X_cat.shape[1]}')
            X_init = X_cat
                    
    else:
        print(f'Numerical data; num: {X_num.shape[1]}')
        X_init = X_num

    # SCALING
    X_init = torch.tensor(scale_data(pd.DataFrame(X_init.float())).values)
    #X_init = scale_data(pd.DataFrame(X_init.float()))
    
    simul_dict = {'X_init': X_init, 
                  'X_num':X_init[:, :d_numerical] if d_numerical != 0 else None, 
                  'X_cat': X_init[:, d_numerical:] if X_cat is not None else None, 
                  'y':y,
                  'task_type':task_type, 
                  'd_numerical': d_numerical}
    
    torch.save(simul_dict, f'{save_dir}/old_proc.pkl')

    data_dict = {'MAR':{'0.3':{}, '0.6': {}}, 
                 'MCAR':{'0.3':{}, '0.6': {}}, 
                 'MNAR': {'0.3':{}, '0.6': {}}}
    
    
    for mechanism in ['MAR', 'MCAR', 'MNAR']:
        
        
        p_miss_r = 0.5
        
        for p_miss_c in [0.3, 0.6]:

            data_dict[mechanism][p_miss_c] = {}
            
            for seed in range(5):

                data_dict[mechanism][p_miss_c][seed] = {}
               
                #seed = 1
                print(mechanism)
                
                set_all_seeds(seed)
                
                # generate mask (missing); True : Miss, False: Obs
                #X_init, X_incomp, mask = simulate_nan(X_init.numpy(), p_miss = p_miss_c, mecha = mechanism, sample_columns = True).values()

                X_init, X_incomp, mask = ampute(pd.DataFrame(X_init), 
                                mechanism = mechanism, 
                                p_miss = p_miss_r, 
                                p_miss_c = p_miss_c ,
                                column_limit = max(round(p_miss_c * X_init.shape[1]), 1), 
                               seed = seed)
                
                
                #print( (1.0*mask).mean(axis = 0) )

                
                X_incomp = torch.tensor(X_init.values).clone().float()
                mask = torch.tensor(mask.values)
                X_init = torch.tensor(X_init.values)

                X_incomp[mask.bool()] = np.nan
                
                X_incomp_num, X_incomp_cat = X_incomp[:, :d_numerical], X_incomp[:, d_numerical:]
                miss_cols = torch.where(mask.float().mean(axis = 0) != 0)[0].tolist()
                fully_obs_cols = [i for i in range(X_init.shape[1]) if i not in miss_cols]
                
                base_dict = {'X_incomp': X_incomp,
                             'mask': mask == 1,
                             'full_cols': fully_obs_cols}

                data_dict[mechanism][p_miss_c][seed] = base_dict

    # SAVE
    torch.save(data_dict, f'{save_dir}/amputed_org.pkl')


---

#### VISUALIZATION

In [ ]:
mech = 'NMAR'
seed = 213
test_missing(X_num, X_cat, mechanism = mech, seed = seed, c = 0, v = 1, var_select = False, standardize = True)
test_missing(X_num, X_cat, mechanism = mech, seed = seed, c = 0, v = 1, var_select = True, standardize = True)
test_missing(X_num, X_cat, mechanism = mech, seed = seed, c = 0, v = 1, var_select = False, standardize = False)
test_missing(X_num, X_cat, mechanism = mech, seed = seed, c = 2, v = 1, var_select = True, standardize = False)

In [ ]:
def test_missing(X_num, X_cat, mechanism = 'MAR', seed = 0, c = 0, v = 1, var_select = False, standardize = True):
    p_miss_r = 0.5
    p_miss_c = 0.8
    #seed = 0
    
    set_all_seeds(seed)
    
    # generate mask (missing); True : Miss, False: Obs
    ps, mask, coeffs = generate_missing(X_num, X_cat, mechanism = mechanism, p = p_miss_r, p_obs_col = 1-p_miss_c, seed = seed, c = c, v = v, var_select = var_select, standardize = standardize)
    
    #print( (1.0*mask).mean(axis = 0) )
    
    X_incomp = X_init.clone()
    X_incomp[mask.bool()] = np.nan
    
    X_incomp_num, X_incomp_cat = X_incomp[:, :d_numerical], X_incomp[:, d_numerical:]
    miss_cols = torch.where(mask.float().mean(axis = 0) != 0)[0].tolist()
    fully_obs_cols = [i for i in range(X_init.shape[1]) if i not in miss_cols]
    
    
    fig, ax = plt.subplots(1, 4, figsize = (16, 4))
    ax_ = ax.flatten()
    
    for idx, axs in enumerate(ax_):
        
        miss_mask = mask[:, miss_cols[idx]]
        obs_mask = mask[:, miss_cols[idx]] == False
        X_obs = X_init[:, miss_cols][obs_mask, idx]
        X_mis = X_init[:, miss_cols][miss_mask, idx]
    
        sns.histplot(X_obs, label = 'obs', bins = 20, stat = 'density', ax = axs)
        sns.histplot(X_mis, label = 'mis', bins = 20, stat = 'density', ax = axs)
    
        axs.set_title(f'var = {idx}')
    plt.suptitle(f'mech = {mechanism}')
    plt.legend()
    plt.show()


In [ ]:
def test_missing(X_num, X_cat, mechanism = 'MAR', seed = 0, c = 0, v = 1, var_select = False, standardize = True):
    
    # generate mask (missing); True : Miss, False: Obs
    ps, mask, coeffs = generate_missing(X_num, X_cat, mechanism = mechanism, p = p_miss_r, p_obs_col = 1-p_miss_c, seed = seed, c = c, v = v, var_select = var_select, standardize = standardize)
    
    #print( (1.0*mask).mean(axis = 0) )
    
    X_incomp = X_init.clone()
    X_incomp[mask.bool()] = np.nan
    
    X_incomp_num, X_incomp_cat = X_incomp[:, :d_numerical], X_incomp[:, d_numerical:]
    miss_cols = torch.where(mask.float().mean(axis = 0) != 0)[0].tolist()
    fully_obs_cols = [i for i in range(X_init.shape[1]) if i not in miss_cols]
    
    
    fig, ax = plt.subplots(1, 4, figsize = (16, 4))
    ax_ = ax.flatten()
    
    for idx, axs in enumerate(ax_):
        
        miss_mask = mask[:, miss_cols[idx]]
        obs_mask = mask[:, miss_cols[idx]] == False
        X_obs = X_init[:, miss_cols][obs_mask, idx]
        X_mis = X_init[:, miss_cols][miss_mask, idx]
    
        sns.histplot(X_obs, label = 'obs', bins = 20, stat = 'density', ax = axs)
        sns.histplot(X_mis, label = 'mis', bins = 20, stat = 'density', ax = axs)
    
        axs.set_title(f'var = {idx}')
    plt.suptitle(f'mech = {mechanism}')
    plt.legend()
    plt.show()
